### Setup

In [1]:
# this first setup block will pretty much always stay the same
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_multiple_reports, do_fit_scenarios
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220714_scenario_projections.ipynb")

2022/07/14 13:37:24 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220628_july_gov_briefing/scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20220628_july_gov_briefing/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-09-18',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/07/14 13:37:31 INFO|-------------------------|{} Retrieving vaccinations data
2022/07/14 13:37:33 INFO|-------------------------|{} Constructing vaccination projections
2022/07/14 13:37:33 INFO|-------------------------|{} Retrieving hospitalizations data
2022/07/14 13:37:35 INFO|----------------------|{} Prepping Model
2022/07/14 13:37:35 INFO|-----------------------|{} Prepping Model
2022/07/14 13:38:27 INFO|----------------------|{} Model prepped for fitting in 52.10940720799999 seconds.
2022/07/14 13:38:27 INFO|----------------------|{} Will fit 25 times
2022/07/14 13:39:40 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 73.09434525 seconds: {0: {'co': 0.8773342495490936}, 14: {'co': 0.6853524265751645}, 28: {'co': 0.5380334941579741}, 42: {'co': 0.0005409486889829486}, 56: {'co': 0.6816118768777373}, 70: {'co': 0.89119421452826}}
2022/07/14 13:39:40 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/07/14 13:39

2022/07/14 13:44:08 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 11.798088624999991 seconds: {504: {'co': 0.7970051383302796}, 518: {'co': 0.7916051680097907}, 532: {'co': 0.7704140650702391}, 546: {'co': 0.7250059903394989}, 560: {'co': 0.7573655134959484}, 574: {'co': 0.7526705140844518}}
2022/07/14 13:44:08 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/07/14 13:44:21 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 12.403755999999987 seconds: {532: {'co': 0.770261719662157}, 546: {'co': 0.7266696538917392}, 560: {'co': 0.7537877960700033}, 574: {'co': 0.7628489955210572}, 588: {'co': 0.7989203481573962}, 602: {'co': 0.7800539538429004}}
2022/07/14 13:44:21 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/07/14 13:44:32 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 9.016813333000016 seconds: {

### Create and run scenarios from Feb 2022 to present

In [ ]:
multiprocess = 6

scenario_params = json.load(open("covid_model/analysis/20220628_july_gov_briefing/scenario_params.json"))

model_args = {
    'base_spec_id': 3214 #model.spec_id # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for beta_mult in [1.0, 0.95, 1.05]:
    for hosp_frac in [0.5, 0.7, 0.9]: # 50%, 70% , or 90% for-COVID
        for school_contact_increase in [1.0, 1.1, 1.2]:
            for immune_escape_mult in [1.0, 0.9, 1.1]:
                hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac), "2022-03-15": (0.34 + 0.66*hosp_frac), "2022-03-30": hosp_frac}
                beta_param_adjustment = [{"param": "betta", "attrs": {"variant": "ba45"}, "mults": {"2020-01-01": beta_mult}, "desc": "sensitivity analysis for beta"}]
                school_contact = [{"param": "betta", "attrs": {'age': '0-19'}, "mults": {"2020-01-01": 1, "2022-08-15": school_contact_increase},"desc": "After August 15th, beta increases by a factor of the school contact increase"}]
                lt5_vacc_adjust = [{"param": "immunity", "attrs": {'age': '0-19', 'vacc': 'shot1'}, "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5, "2022-06-30": 0.98 + 0.02*vacc_eff_lt5, "2022-07-10": 0.97 + 0.03*vacc_eff_lt5}}]
                immune_mult = [{"param": "immune_escape", "from_attrs":{"variant": ["omicron", "ba2", "ba2121"]}, "to_attrs": {"variant": ["ba45"]}, "mults": {"2020-01-01": immune_escape_mult},  "desc": "sensitivity analysis for immune escape"}]
                scenario_model_args.append({'params_defs': scenario_params + beta_param_adjustment + school_contact + lt5_vacc_adjust + immune_mult,
                                            'hosp_reporting_frac': hrf,
                                            'tags': {'hosp_frac': hosp_frac, 'school_contact_increase': school_contact_increase, 'immune_escape_mult': immune_escape_mult}})
            
            

In [ ]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

### Run the report for each fit model

In [ ]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=True, solve_model=True, immun_variants=['ba2121', 'ba45'], from_date='2022-01-01')

In [ ]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-09-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')